# Regression test suite for the Harmony Subsetter with Multi-dimensional Concatenator:

<!-- This notebook provides condensed examples of using Harmony to make requests against the Variable Subsetter services developed and managed by the Data Services team on the Transformation Train. This service makes use of CF-Conventions to retrieve all requested variable from OPeNDAP, along with all those other variables required to make the output product usable in downstream processing (e.g., coordinate and dimension variables). This service can be used with any OPeNDAP-enabled collection that adheres to the Climate and Forecast metadata conventions. -->

The data retrieved from the service chain will be in a NetCDF-4 format.

## Prerequisites

<!-- The dependencies for this notebook are listed in the environment.yaml. To test or install locally, create the papermill environment used in the automated regression testing suite:

conda env create -f ./environment.yaml && conda activate papermill-variable-subsetter -->

A .netrc file must also be located in the test directory of this repository.

## Import requirements:

In [1]:
from os.path import exists

from harmony import BBox, Client, Collection, Environment, Request
import numpy as np

from utilities import (
    compare_results_to_reference_file, print_success, 
    compare_global_attributes_to_reference,
    remove_results_files, submit_and_download
)

In [2]:
harmony_host_url = 'https://harmony.uat.earthdata.nasa.gov'

## Identify Harmony environment (for easier reference):

In [3]:
host_environment = {'http://localhost:3000': Environment.LOCAL,
                    'https://harmony.sit.earthdata.nasa.gov': Environment.SIT,
                    'https://harmony.uat.earthdata.nasa.gov': Environment.UAT,
                    'https://harmony.earthdata.nasa.gov': Environment.PROD}

harmony_environment = host_environment.get(harmony_host_url)

if harmony_environment is not None:
    harmony_client = Client(env=harmony_environment)

# Begin regression tests:

## Concatenation Service Chain

The Subsetter with Multi-dimensional Concatenator is currently only activated for collections in the UAT environment. Requests will be made against granules in the TEMPO L2 $NO_2$ and Formaldehyde collections, as these two are the only currently associated collections.

In [4]:
concatenator_chain_non_prod_information = {'collection': 'C1254854453-LARC_CLOUD'}

concatenator_chain_env = {Environment.UAT: concatenator_chain_non_prod_information}

if harmony_environment in concatenator_chain_env:
    concatenator_chain_info = concatenator_chain_env[harmony_environment]
else:
    concatenator_chain_info = None

## Concatenator request:

This is a request to retrieve a concatenated file from two TEMPO granules. The request will utilize L2ss-py, Batchee, STITCHEE, and CONCISE to subset spatially, and then both extend the existing `mirror_step` dimension and concatenate along a new `subset_index` dimension.

In [5]:
concatenator_chain_output_file_name = 'subset-concatenator-3-files-3-vars-spatialsubset_result.nc4'

In [6]:
if concatenator_chain_info is not None:
    concatenator_chain_request = Request(
        collection=Collection(id=concatenator_chain_info['collection']),
        concatenate="True",
        spatial=BBox(-100, 35, -90, 40),
        variables=['/product/vertical_column_total', 
                   '/product/vertical_column_troposphere',
                   '/product/vertical_column_stratosphere'],
        extend="mirror_step",
        max_results=3
    )
    assert concatenator_chain_request.is_valid()

    submit_and_download(harmony_client, concatenator_chain_request, concatenator_chain_output_file_name)
    assert exists(concatenator_chain_output_file_name), 'Unsuccessful Subsetter-Concatenation request.'

    # Check results
    reference_file_name = 'reference_files/' + concatenator_chain_output_file_name
    compare_results_to_reference_file(concatenator_chain_output_file_name, reference_file_name,
                                      "/")
    compare_results_to_reference_file(concatenator_chain_output_file_name, reference_file_name,
                                      "product")
    compare_results_to_reference_file(concatenator_chain_output_file_name, reference_file_name,
                                      "geolocation")
    compare_global_attributes_to_reference(concatenator_chain_output_file_name, reference_file_name)
    
    
    print_success('Subsetter-Concatenation request.')
else:
    print(f'The Subsetter-Concatenation is not configured for environment: "{harmony_environment}" - skipping test.')

job-id: 92e9101b-64a5-4cb2-8d90-2391e1f58878
C1254854453-LARC_CLOUD_merged.nc4
Downloaded: C1254854453-LARC_CLOUD_merged.nc4
Saved output to: subset-concatenator-3-files-3-vars-spatialsubset_result.nc4
Success: Subsetter-Concatenation request.


In [7]:
remove_results_files()